In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pandas as pd
import triage
from triage.component.audition import Auditioner
from triage.component.audition.pre_audition import PreAudition
from triage.component.audition.rules_maker import SimpleRuleMaker, RandomGroupRuleMaker, create_selection_grid

from matplotlib import pyplot as plt

In [3]:
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432
%env DEMO_DATABASE_USER=postgres
%env DEMO_DATABASE_PASS=postgres

env: DEMO_DATABASE_NAME=donors_choose
env: DEMO_DATABASE_HOST=localhost
env: DEMO_DATABASE_PORT=5432
env: DEMO_DATABASE_USER=postgres
env: DEMO_DATABASE_PASS=postgres


In [4]:
!psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'
!psql -U postgres -c 'CREATE DATABASE donors_choose;'

ALTER ROLE
ERROR:  database "donors_choose" is being accessed by other users
DETAIL:  There are 2 other sessions using the database.
ERROR:  database "donors_choose" already exists


In [5]:
db_url = URL(
            'postgres',
            host='localhost',
            username='postgres',
            database='donors_choose',
            password='postgres',
            port=5432,
        )

db_engine = create_engine(db_url)

In [6]:
!curl -s -OL https://dsapp-public-data-migrated.s3.us-west-2.amazonaws.com/donors_sampled_20210920_v3.dmp

In [7]:
!PGPASSWORD=$DEMO_DATABASE_PASS pg_restore -h $DEMO_DATABASE_HOST -p $DEMO_DATABASE_PORT -d $DEMO_DATABASE_NAME -U $DEMO_DATABASE_USER -O -j 8 donors_sampled_20210920_v3.dmp

pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 5; 2615 8563827 SCHEMA data dssg_admin
pg_restore: error: could not execute query: ERROR:  schema "data" already exists
Command was: CREATE SCHEMA data;


pg_restore: from TOC entry 197; 1259 8563828 TABLE donations dssg_admin
pg_restore: error: could not execute query: ERROR:  relation "donations" already exists
Command was: CREATE TABLE data.donations (
    donationid character varying(50),
    projectid_str character varying(50),
    donor_acctid character varying(50),
    donor_city character varying(50),
    donor_state character varying(10),
    donor_zip character varying(10),
    is_teacher_acct boolean,
    donation_timestamp timestamp without time zone,
    donation_to_project numeric,
    donation_optional_support numeric,
    donation_total numeric,
    dollar_amount character varying(50),
    donation_included_optional_support boolean,
    payment_method character varying(100),
    payment_included_acct_creadit b

In [8]:
config_yaml = """
config_version: 'v8'

model_comment: 'triage demo'

random_seed: 1995

temporal_config:

    # first date our feature data is good
    feature_start_time: '2000-01-01'
    feature_end_time: '2013-06-01'

    # first date our label data is good
    # donorschoose: as far back as we have good donation data
    label_start_time: '2011-09-02'
    label_end_time: '2013-06-01'

    model_update_frequency: '4month'

    # length of time defining a test set
    test_durations: ['3month']
    # defines how far back a training set reaches
    max_training_histories: ['1y']

    # we sample every day, since new projects are posted
    # every day
    training_as_of_date_frequencies: ['1day']
    test_as_of_date_frequencies: ['1day']

    # when posted project timeout
    label_timespans: ['3month']


label_config:
  query: |
    WITH cohort_query AS (
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
    )
    , cohort_donations AS (
      SELECT
        c.entity_id,
        COALESCE(SUM(d.donation_to_project), 0) AS total_donation
      FROM cohort_query c
      LEFT JOIN data.donations d
        ON c.entity_id = d.entity_id
        AND d.donation_timestamp
          BETWEEN '{as_of_date}'::date - interval '1day'
          AND '{as_of_date}'::date + interval '{label_timespan}'
      GROUP BY 1
    )
    SELECT c.entity_id,
    CASE
      WHEN COALESCE(d.total_donation, 0) >= p.total_asking_price THEN 0
      ELSE 1
    END AS outcome
    FROM cohort_query c
    JOIN data.projects p USING(entity_id)
    LEFT JOIN cohort_donations d using(entity_id)

  name: 'fully_funded'


feature_aggregations:
  -
    prefix: 'project_features'
    from_obj: 'data.projects'
    knowledge_date_column: 'date_posted'

    aggregates_imputation:
      all:
        type: 'zero'

    categoricals_imputation:
      all:
        type: 'null_category'

    categoricals:
      -
        column: 'resource_type'
        metrics:
          - 'max'
        choice_query: 'select distinct resource_type from data.projects'

    aggregates:
      -
        quantity: 'total_asking_price'
        metrics:
          - 'sum'

    # Since our time-aggregate features are precomputed, feature interval is
    # irrelvant. We keep 'all' as a default.
    intervals: ['all']

grid_config:
    'sklearn.ensemble.RandomForestClassifier':
        n_estimators: [100]
        max_depth: [50, 75]
        min_samples_split: [25, 15]

    'sklearn.tree.DecisionTreeClassifier':
        max_depth: [3]
        max_features: [null]
        min_samples_split: [25]

    'triage.component.catwalk.estimators.classifiers.ScaledLogisticRegression':
        C: [0.1]
        penalty: ['l1']

    'triage.component.catwalk.baselines.rankers.BaselineRankMultiFeature':
        rules:
            - [{feature: 'project_features_entity_id_all_total_asking_price_sum', low_value_high_score: False}]


scoring:
    testing_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9,
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]

    training_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9,
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]

bias_audit_config:
    from_obj_table: 'data.projects'
    attribute_columns:
        - 'teacher_prefix'
    knowledge_date_column: 'date_posted'
    entity_id_column: 'entity_id'
    ref_groups_method: 'predefined'
    ref_groups:
        'teacher_prefix': 'Mr.'
    thresholds:
        percentiles: [5, 10, 15, 20, 25, 50, 100]
        top_n: [25, 50, 100]

individual_importance:
    methods: [] # empty list means don't calculate individual importances
    n_ranks: 1
"""

In [9]:
database_yaml = """
host: localhost
user: postgres
db: donors_choose
pass: postgres
port: 5432
role: postgres
"""

In [10]:
import yaml

from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
from triage.experiments import MultiCoreExperiment
import logging

import os

from sqlalchemy.event import listens_for
from sqlalchemy.pool import Pool

def run_triage():

  # andrew_id = os.getenv('USER')
  # user_path = os.path.join('/data/users/', andrew_id)
  user_path = '/tmp/content'

  # creating database engine
  # dbfile = os.path.join(user_path, 'database.yaml')

  # with open(dbfile, 'r') as dbf:
  #     dbconfig = yaml.safe_load(dbf)

  dbconfig = yaml.safe_load(database_yaml)
  print(dbconfig['role'])

  # assume group role to ensure shared permissions
  @listens_for(Pool, "connect")
  def assume_role(dbapi_con, connection_record):
      logging.debug(f"setting role {dbconfig['role']};")
      dbapi_con.cursor().execute(f"set role {dbconfig['role']};")
      # logging.debug(f"setting role postres;")
      # dbapi_con.cursor().execute(f"set role postgres;")

  db_url = URL(
              'postgres',
              host=dbconfig['host'],
              username=dbconfig['user'],
              database=dbconfig['db'],
              password=dbconfig['pass'],
              port=dbconfig['port'],
          )

  db_engine = create_engine(db_url)

  triage_output_path = os.path.join(user_path, 'triage_output')
  os.makedirs(triage_output_path, exist_ok=True)

  # loading config file
  # with open('%s_triage_config.yaml' % andrew_id, 'r') as fin:
  #     config = yaml.safe_load(fin)

  config = yaml.safe_load(config_yaml)

  # creating experiment object
  experiment = MultiCoreExperiment(
      config = config,
      db_engine = db_engine,
      project_path = triage_output_path,
      n_processes=2,
      n_bigtrain_processes=1,
      n_db_processes=2,
      replace=True,
      save_predictions=True
      )

  # experiment.validate()
  experiment.run()

In [11]:
run_triage()

postgres
2023-10-23 23:40:40 - VERBOSE Matrices and trained models will be saved in /tmp/content/triage_output
2023-10-23 23:40:40 -  NOTICE Replace flag is set to true. Matrices, models, evaluations and predictions (if they exist) will be replaced
2023-10-23 23:40:40 - VERBOSE Using random seed [1995] for running the experiment
2023-10-23 23:40:41 -    INFO cohort_config missing or unrecognized, but labels are configured. Labels will be used as the cohort.
2023-10-23 23:40:41 -  NOTICE scoring.subsets missing in the configuration file or unrecognized. No subsets will be generated
2023-10-23 23:40:41 - SUCCESS Experiment validation ran to completion with no errors
2023-10-23 23:40:41 - VERBOSE Computed and stored temporal split definitions
2023-10-23 23:40:41 -    INFO Setting up labels
2023-10-23 23:41:16 - SUCCESS Labels set up in the table labels_fully_funded_812318d85a3194ad2c0d22092d6a92d6 successfully 
2023-10-23 23:41:16 -    INFO Setting up cohort
2023-10-23 23:41:16 - SUCCESS 


KeyboardInterrupt



2023-10-23 23:44:21 -    INFO getcrosstabs: attribute columns to perform crosstabs:teacher_prefix
get_disparity_predefined_group()
2023-10-23 23:44:22 -    INFO get_group_value_fairness...
2023-10-23 23:44:22 -    INFO getcrosstabs: attribute columns to perform crosstabs:teacher_prefix
get_disparity_predefined_group()
2023-10-23 23:44:22 -    INFO get_group_value_fairness...
2023-10-23 23:44:22 -    INFO Model 5 evaluation on test matrix a81c0dc805971d5bc9104d35826100d9 completed.
2023-10-23 23:44:23 -    INFO Found old predictions for model 5 on train matrix 7be1c9af421575360abd144c37e932d9. Those predictions were deleted.
2023-10-23 23:44:30 -    INFO getcrosstabs: attribute columns to perform crosstabs:teacher_prefix
get_disparity_predefined_group()
2023-10-23 23:44:31 -    INFO get_group_value_fairness...
2023-10-23 23:44:31 -    INFO getcrosstabs: attribute columns to perform crosstabs:teacher_prefix
get_disparity_predefined_group()
2023-10-23 23:44:31 -    INFO get_group_value_fa

In [33]:
plt.style.use('ggplot')
%matplotlib inline
pd.set_option('precision', 4)

metric = 'precision@'
parameter = '10_pct'

dbconfig = yaml.safe_load(database_yaml)
db_url = URL(
            'postgres',
            host=dbconfig['host'],
            username=dbconfig['user'],
            database=dbconfig['db'],
            password=dbconfig['pass'],
            port=dbconfig['port'],
        )

conn = create_engine(db_url)

best_dist_table = 'audition_best_dist'

In [35]:
run_hash = \
pd.read_sql('''
  SELECT *
  FROM triage_metadata.triage_runs
  ORDER BY last_updated_time DESC
  LIMIT 1
''', db_engine)['run_hash'].reset_index(drop=True).loc[0]

In [36]:
pre_aud = PreAudition(
    conn,
    baseline_model_types=[
        'sklearn.dummy.DummyClassifier',
        'triage.component.catwalk.baselines.rankers.BaselineRankMultiFeature',
        'triage.component.catwalk.baselines.thresholders.SimpleThresholder'
    ]
)

# select model groups by experiment hash id
model_groups = pre_aud.get_model_groups_from_experiment(run_hash)

# Note that this will find train_end_times associated with the model groups defined above
end_times = pre_aud.get_train_end_times(after='1900-01-01')

In [37]:
aud = Auditioner(
    db_engine = conn,
    model_group_ids = model_groups['model_groups'],
    train_end_times = end_times,
    initial_metric_filters = [{'metric': metric, 'parameter': parameter, 'max_from_best': 1.0, 'threshold_value': 0.0}],
    distance_table = best_dist_table,
    baseline_model_group_ids = model_groups['baseline_model_groups'] # optional
)